# ST2195 Coursework

#### Student Number : 210421794

## Question 5 : Use the available variables to construct a model that predicts delays

In [1]:
# Import and load packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.compose import ColumnTransformer

In [2]:
# Load the datasets

flight2005 = pd.read_csv("/Users/jessicalawrence/Desktop/ST2195 Coursework Project/2005.csv")
flight2006 = pd.read_csv("/Users/jessicalawrence/Desktop/ST2195 Coursework Project/2006.csv")

airports = pd.read_csv("/Users/jessicalawrence/Desktop/ST2195 Coursework Project/airports.csv")
carriers = pd.read_csv("/Users/jessicalawrence/Desktop/ST2195 Coursework Project/carriers.csv")
plane_data = pd.read_csv("/Users/jessicalawrence/Desktop/ST2195 Coursework Project/plane-data.csv")

In [3]:
# Combine the data from 2005 and 2006

flights = pd.concat([flight2005, flight2006])
flights

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,2005,1,28,5,1603.0,1605,1741.0,1759,UA,541,...,4,23,0,NaN,0,0,0,0,0,0
1,2005,1,29,6,1559.0,1605,1736.0,1759,UA,541,...,6,15,0,NaN,0,0,0,0,0,0
2,2005,1,30,7,1603.0,1610,1741.0,1805,UA,541,...,9,18,0,NaN,0,0,0,0,0,0
3,2005,1,31,1,1556.0,1605,1726.0,1759,UA,541,...,11,10,0,NaN,0,0,0,0,0,0
4,2005,1,2,7,1934.0,1900,2235.0,2232,UA,542,...,5,10,0,NaN,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7141917,2006,12,29,5,1246.0,1249,1452.0,1459,DL,1675,...,6,12,0,NaN,0,0,0,0,0,0
7141918,2006,12,29,5,1225.0,1155,2033.0,1931,DL,1676,...,12,35,0,NaN,0,30,0,32,0,0
7141919,2006,12,29,5,2118.0,2115,2254.0,2241,DL,1676,...,9,27,0,NaN,0,0,0,0,0,0
7141920,2006,12,29,5,2122.0,2127,2209.0,2223,DL,1677,...,5,21,0,NaN,0,0,0,0,0,0


In [4]:
# Remove rows with missing values

flights.dropna(subset = ['DepTime', 'ArrTime', 'ArrDelay', 'DepDelay'], inplace = True)

In [5]:
# Re-check the null values

flights.isnull().sum()

Year                        0
Month                       0
DayofMonth                  0
DayOfWeek                   0
DepTime                     0
CRSDepTime                  0
ArrTime                     0
CRSArrTime                  0
UniqueCarrier               0
FlightNum                   0
TailNum                     0
ActualElapsedTime           0
CRSElapsedTime              0
AirTime                     0
ArrDelay                    0
DepDelay                    0
Origin                      0
Dest                        0
Distance                    0
TaxiIn                      0
TaxiOut                     0
Cancelled                   0
CancellationCode     13996639
Diverted                    0
CarrierDelay                0
WeatherDelay                0
NASDelay                    0
SecurityDelay               0
LateAircraftDelay           0
dtype: int64

In [6]:
# Left join flights dataset with plane_data dataset

flights_merge = pd.merge(flights, plane_data, left_on='TailNum', right_on='tailnum', how='left')
flights_merge

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,LateAircraftDelay,tailnum,type,manufacturer,issue_date,model,status,aircraft_type,engine_type,year
0,2005,1,28,5,1603.0,1605,1741.0,1759,UA,541,...,0,N935UA,Corporation,BOEING,06/29/1992,737-522,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1992
1,2005,1,29,6,1559.0,1605,1736.0,1759,UA,541,...,0,N941UA,Corporation,BOEING,10/07/1992,737-522,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1992
2,2005,1,30,7,1603.0,1610,1741.0,1805,UA,541,...,0,N342UA,Corporation,BOEING,11/28/1988,737-322,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1988
3,2005,1,31,1,1556.0,1605,1726.0,1759,UA,541,...,0,N326UA,Corporation,BOEING,09/11/2007,737-322,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1988
4,2005,1,2,7,1934.0,1900,2235.0,2232,UA,542,...,0,N902UA,Corporation,BOEING,04/30/1993,737-522,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13996635,2006,12,29,5,1246.0,1249,1452.0,1459,DL,1675,...,0,N905DE,Corporation,MCDONNELL DOUGLAS CORPORATION,06/03/1992,MD-88,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1992
13996636,2006,12,29,5,1225.0,1155,2033.0,1931,DL,1676,...,0,N651DL,Corporation,BOEING,07/28/1989,757-232,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1989
13996637,2006,12,29,5,2118.0,2115,2254.0,2241,DL,1676,...,0,N143DA,Corporation,BOEING,10/16/1998,767-332,Valid,Fixed Wing Multi-Engine,Turbo-Fan,1998
13996638,2006,12,29,5,2122.0,2127,2209.0,2223,DL,1677,...,0,N904DA,Corporation,MCDONNELL DOUGLAS,05/15/1995,MD-90-30,Valid,Fixed Wing Multi-Engine,Turbo-Jet,1995


In [7]:
# Removing any invalid values in 'year' column and convert its values to integer

flights_merge = flights_merge[flights_merge['year'] != '0000']
flights_merge = flights_merge[flights_merge['year'] != 'None']

flights_merge = flights_merge.loc[flights_merge['year'].notna()]
flights_merge['year'] = flights_merge['year'].astype(int)

flights_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10223636 entries, 0 to 13996639
Data columns (total 38 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Year               int64  
 1   Month              int64  
 2   DayofMonth         int64  
 3   DayOfWeek          int64  
 4   DepTime            float64
 5   CRSDepTime         int64  
 6   ArrTime            float64
 7   CRSArrTime         int64  
 8   UniqueCarrier      object 
 9   FlightNum          int64  
 10  TailNum            object 
 11  ActualElapsedTime  float64
 12  CRSElapsedTime     float64
 13  AirTime            float64
 14  ArrDelay           float64
 15  DepDelay           float64
 16  Origin             object 
 17  Dest               object 
 18  Distance           int64  
 19  TaxiIn             int64  
 20  TaxiOut            int64  
 21  Cancelled          int64  
 22  CancellationCode   object 
 23  Diverted           int64  
 24  CarrierDelay       int64  
 25  WeatherDelay    

In [8]:
# Create a new column 'plane_age' that indicates the age of plane
flights_merge['plane_age'] = flights_merge['Year'] - flights_merge['year']

#Filter rows with age of plane > 0
flights_merge = flights_merge[flights_merge['plane_age'] >= 0]

### Split data into train and test set

In [9]:
# Querying numerical variables that are required for building the future prediction (i.e. Subsetting the dataset by its features & the target)

numerical_features = ["CRSDepTime", "CRSArrTime", "CRSElapsedTime", "Year", "Month", "DayofMonth", "DayOfWeek", "plane_age"]

x = flights_merge[numerical_features].copy()
y = flights_merge["ArrDelay"].copy()

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,random_state = 1)

### Pipelines: Pre-Processing State

In [11]:
# Applying SimpleImputer and StandardScaler into a pipeline

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())])

data_transformer = ColumnTransformer(
    transformers = [('numerical', numerical_transformer, numerical_features)])

### Linear Regression Model

In [12]:
pipe_lm = Pipeline(steps=[('data_transformer', data_transformer),
                          ('pipe_ln', LinearRegression())])

In [13]:
param_grid = {
    'data_transformer__numerical__imputer__strategy': ['mean', 'median'],
}

In [14]:
grid_lm = GridSearchCV(pipe_lm, param_grid=param_grid)
grid_lm.fit(x_train, y_train);

In [15]:
y_predict_test_lm = grid_lm.predict(x_test)
y_predict_train_lm = grid_lm.predict(x_train)

In [16]:
mse_test_lm = mean_squared_error(y_test, y_predict_test_lm) # Calculating Mean Squared Error (MSE) for test error
mse_train_lm = mean_squared_error(y_train, y_predict_train_lm) # Calculating Mean Squared Error (MSE) for train error
r_squared_lm = r2_score(y_test, y_predict_test_lm) # Calculating R-Squared (The closer the value to 1, the better the model)

In [17]:
# Create a new dataframe with the values

lm_model = pd.DataFrame({
    'Metrics': ['MSE on train data', 'MSE on test data', 'R Squared'],
    'Values': [mse_train_lm, mse_test_lm, r_squared_lm]
})

lm_model

,Metrics,Values
0,MSE on train data,1271.811629
1,MSE on test data,1265.301839
2,R Squared,0.021844


### Ridge Regression Model

In [18]:
pipe_ridge = Pipeline(steps=[('data_transformer', data_transformer),
                             ('pipe_ridge', Ridge(random_state=1))])

In [19]:
grid_ridge = GridSearchCV(pipe_ridge, param_grid=param_grid)
grid_ridge.fit(x_train, y_train);

In [20]:
y_predict_test_ridge = grid_ridge.predict(x_test)
y_predict_train_ridge = grid_ridge.predict(x_train)

In [21]:
mse_test_ridge = mean_squared_error(y_test, y_predict_test_ridge) # Calculating Mean Squared Error (MSE) for test error
mse_train_ridge = mean_squared_error(y_train, y_predict_train_ridge) # Calculating Mean Squared Error (MSE) for train error
r_squared_ridge = r2_score(y_test, y_predict_test_ridge) # Calculating R-Squared (The closer the value to 1, the better the model)

In [22]:
# Create a new dataframe with the values

ridge_model = pd.DataFrame({
    'Metrics': ['MSE on train data', 'MSE on test data', 'R Squared'],
    'Values': [mse_train_ridge, mse_test_ridge, r_squared_ridge]
})

ridge_model

,Metrics,Values
0,MSE on train data,1271.811629
1,MSE on test data,1265.301839
2,R Squared,0.021844


### Lasso Regression Model

In [23]:
pipe_lasso = Pipeline(steps=[('data_transformer', data_transformer),
                             ('pipe_lasso', Lasso(random_state=1))])

In [24]:
grid_lasso = GridSearchCV(pipe_lasso, param_grid=param_grid)
grid_lasso.fit(x_train, y_train);

In [25]:
y_predict_test_lasso = grid_lasso.predict(x_test)
y_predict_train_lasso = grid_lasso.predict(x_train)

In [26]:
mse_test_lasso = mean_squared_error(y_test, y_predict_test_lasso) # Calculating Mean Squared Error (MSE) for test error
mse_train_lasso = mean_squared_error(y_train, y_predict_train_lasso) # Calculating Mean Squared Error (MSE) for train error
r_squared_lasso = r2_score(y_test, y_predict_test_lasso) # Calculating R-Squared (The closer the value to 1, the better the model)

In [27]:
# Create a new dataframe with the values
lasso_model = pd.DataFrame({
    'Metrics': ['MSE on train data', 'MSE on test data', 'R Squared'],
    'Values': [mse_train_lasso, mse_test_lasso, r_squared_lasso]
})

# Display the dataframe
lasso_model

,Metrics,Values
0,MSE on train data,1274.907379
1,MSE on test data,1268.264237
2,R Squared,0.019554


### Random Forest using sample

In [28]:
# Randomnly select 100,000 rows from the flights dataframe

sample_flights = flights_merge.sample(n=100000, random_state=1, replace = False)

In [29]:
numerical_features = ["CRSDepTime", "CRSArrTime", "CRSElapsedTime", "Year", "Month", "DayofMonth", "DayOfWeek", "plane_age"]

x = sample_flights[numerical_features].copy()
y = sample_flights["ArrDelay"].copy()

In [30]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3,random_state = 1)

In [31]:
# Applying SimpleImputer and StandardScaler into a pipeline

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler())])

data_transformer = ColumnTransformer(
    transformers = [('numerical', numerical_transformer, numerical_features)])

In [32]:
pipe_rf = Pipeline(steps=[('data_transformer', data_transformer),
                          ('pipe_rf', RandomForestRegressor(random_state=1))])

In [33]:
param_grid = {
    'data_transformer__numerical__imputer__strategy': ['mean', 'median'],
}

In [34]:
grid_rf = GridSearchCV(pipe_rf, param_grid=param_grid, cv=5, n_jobs=-1)
grid_rf.fit(x_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('data_transformer',
                                        ColumnTransformer(transformers=[('numerical',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer()),
                                                                                         ('scaler',
                                                                                          StandardScaler())]),
                                                                         ['CRSDepTime',
                                                                          'CRSArrTime',
                                                                          'CRSElapsedTime',
                                                                          'Year',
                                                                 

In [35]:
y_predict_test_rf = grid_rf.predict(x_test)
y_predict_train_rf = grid_rf.predict(x_train)

In [36]:
mse_test_rf = mean_squared_error(y_test, y_predict_test_rf) # Calculating Root Mean Squared Error (RMSE) for test error
mse_train_rf = mean_squared_error(y_train, y_predict_train_rf) # Calculating Root Mean Squared Error (RMSE) for train error
r_squared_rf = r2_score(y_test, y_predict_test_rf) # Calculating R-Squared (The closer the value to 1, the better the model)

In [37]:
# Create a new dataframe with the values
rf_model = pd.DataFrame({
    'Metrics': ['MSE on train data', 'MSE on test data', 'R Squared'],
    'Values': [mse_train_rf, mse_test_rf, r_squared_rf]
})

# Display the dataframe
rf_model

,Metrics,Values
0,MSE on train data,181.278852
1,MSE on test data,1306.692562
2,R Squared,-0.016161


### Comparing MSE & R Squared Between Linear Regression, Ridge Regression, Lasso Regression, and Random Forest

In [38]:
model_compare = pd.DataFrame({"":["MSE on train data", "MSE on test data", "R Squared"],
                                "Linear Regression":[mse_train_lm, mse_test_lm, r_squared_lm],
                                "Ridge Regression":[mse_train_ridge, mse_test_ridge, r_squared_ridge],
                                "Lasso Regression":[mse_train_lasso, mse_test_lasso, r_squared_lasso],
                                "Random Forest":[mse_train_rf, mse_test_rf, r_squared_rf]})
model_compare

,,Linear Regression,Ridge Regression,Lasso Regression,Random Forest
0,MSE on train data,1271.811629,1271.811629,1274.907379,181.278852
1,MSE on test data,1265.301839,1265.301839,1268.264237,1306.692562
2,R Squared,0.021844,0.021844,0.019554,-0.016161
